In [1]:
import time
from web3 import Web3, constants
from config import *
import math
from datetime import date
from math_utils import *
from network_utils import *
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
filepath = "data/stETH/"

In [3]:
import time
import datetime

timestamp = int(time.mktime(datetime.datetime.strptime("17/07/2023", "%d/%m/%Y").timetuple()))

block_number = get_block_by_timestamp_eth(int(timestamp))

In [4]:
block_number

17707194

In [5]:
with open("STETH-26DEC2024.json", 'r') as f:
  pool = json.load(f)

pool

{'name': 'SY stETH',
 'SY': '0xcbC72d92b2dc8187414F6734718563898740C0BC',
 'YT': '0xc3863CCcd012f8E45D72Ec87c5A9C4F77e1C7549',
 'PT': '0x7758896b6AC966BbABcf143eFA963030f17D3EdF',
 'market': '0xD0354D4e7bCf345fB117cabe41aCaDb724eccCa2',
 'expiry': 1735171200,
 'scalarRoot': '76568950000000000000',
 'initAnchor': '1104003000000000000'}

In [6]:
abi_url = f'https://api.etherscan.io/api?module=contract&action=getabi&address={pool["YT"]}&format=raw'
abi = requests.get(abi_url).text

YT_contract = web3_ETH.eth.contract(pool["YT"], abi=abi)
index = to_normal_float(YT_contract.functions.pyIndexStored().call(block_identifier=block_number))
print(index)

1.132254404648125


In [8]:
YT_contract = get_contract_eth(address=pool["YT"])
index = to_normal_float(YT_contract.functions.pyIndexStored().call(block_identifier=block_number))
print(index)

time.sleep(5)
market_contract = get_contract_eth(address=pool["market"])

time.sleep(5)
SY_contract = get_contract_eth(address=pool["SY"])

1.132254404648125


In [9]:
timestamp = int(time.mktime(datetime.datetime.strptime("16/07/2023", "%d/%m/%Y").timetuple()))
data_onchain = get_info(SY_contract, YT_contract, market_contract, timestamp, 'eth')
data_onchain


This SY has no reward.


{'date': '16-07-23',
 'asset': 5475.626268503404,
 'PT': 1706.1881085480825,
 'LP': 3426.534417543608,
 'SY': 4836.536937956021,
 'exchange_rate': 1.0634329047497038,
 'X_reward_index': 0,
 'PENDLE_reward_index': 613.9515378777837,
 'average_boost_factor': 1.6878209402096314,
 'SY_index': 1.1321377958538803}

In [10]:
import pandas as pd

data_df = pd.DataFrame()

timestamp_start = int(time.mktime(datetime.datetime.strptime("24/04/2023", "%d/%m/%Y").timetuple()))
timestamp_end = int(time.mktime(datetime.datetime.strptime("24/07/2023", "%d/%m/%Y").timetuple()))

for i in range(timestamp_start, timestamp_end, DAY_DELTA * DAY):
    data_onchain = pd.DataFrame(get_info(SY_contract, YT_contract, market_contract, i, 'eth'), index=[0])
    data_df = pd.concat([data_df, data_onchain], ignore_index=True)

This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.
This SY has no reward.


In [11]:
n = len(data_df)
for i in range(n):
    print(data_df.loc[i, "PT"]/(data_df.loc[i, "exchange_rate"]*data_df.loc[i, "LP"]) + data_df.loc[i, "asset"]/data_df.loc[i, "LP"])

2.046711752047126
2.0472804907149067
2.0477663569368527
2.048654292646775
2.0499963943109916
2.0516982120697103
2.052787797317024
2.0534262132193053
2.0541284358896377
2.0554941159432776
2.0560072040549673
2.058209381774073
2.0586878102059534
2.0597219512254306
2.0600161377986517
2.057511819542649
2.060042036090799
2.0625152690150887
2.063570794991378
2.0642601930977835
2.0656445992764545
2.066460485421627
2.0675606712693173


In [12]:
IL_A1 = IL(data_df, 'A', False, ETH, 0)
IL_A2 = IL(data_df, 'A', True, ETH, 0)
IL_B1 = IL(data_df, 'B', False, ETH, 0)
IL_B2 = IL(data_df, 'B', True, ETH, 0)
IL_C1 = IL(data_df, 'C', False, ETH, 0)
IL_C2 = IL(data_df, 'C', True, ETH, 0)

In [13]:
write_array_to_csv(IL_A1, filepath + "IL_A1.csv", timestamp_start)
write_array_to_csv(IL_A2, filepath + "IL_A2.csv", timestamp_start)
write_array_to_csv(IL_B1, filepath + "IL_B1.csv", timestamp_start)
write_array_to_csv(IL_B2, filepath + "IL_B2.csv", timestamp_start)
write_array_to_csv(IL_C1, filepath + "IL_C1.csv", timestamp_start)
write_array_to_csv(IL_C2, filepath + "IL_C2.csv", timestamp_start)

In [14]:
LP_profit_PENDLE = in_pool_value(data_df, asset_price=ETH, X_price=0, PENDLE_incentive=True, with_YT = False)
LP_profit = in_pool_value(data_df, asset_price=ETH, X_price=0, PENDLE_incentive=False, with_YT = False)
LP_profit_PENDLE_YT = in_pool_value(data_df, asset_price=ETH, X_price=0, PENDLE_incentive=True, with_YT = True)
LP_profit_YT = in_pool_value(data_df, asset_price=ETH, X_price=0, PENDLE_incentive=False, with_YT = True)

[1.0, 1.0089900035618553, 1.017078155974308, 1.0239057633090507, 1.027579407856493, 1.035388204586292, 1.0407177980782718, 1.0430169647303862, 1.047590131451408, 1.051166179499191, 1.0549139938876348, 1.0598219670657116, 1.06763744365093, 1.0740097144194551, 1.0773843133532846, 1.087542177243934, 1.0919659198902532, 1.0999823213774413, 1.1035590447008592, 1.1050574884098037, 1.1071653095641905, 1.107170842613554, 1.1081905553284046]
[<NA>, 1.0, 1.008033906569677, 1.0147375460071404, 1.018432427223637, 1.0260936996916896, 1.0314341278113162, 1.0341830331232325, 1.0386602815736934, 1.0425197886735262, 1.0462339614901228, 1.0511441620979045, 1.0583328167339365, 1.064320938154537, 1.0677075654904502, 1.07688718379691, 1.081306550533708, 1.088817777244383, 1.0923256340515408, 1.0938654221535202, 1.0959916960415443, 1.0961538752586975, 1.0972199446537945]
[<NA>, <NA>, 1.0, 1.0065934498729978, 1.0103072552569865, 1.0178373968303356, 1.0231875139010305, 1.0263364393010987, 1.030728423247695, 1

In [15]:
write_array_to_csv(LP_profit_PENDLE, "data\LP_profit\stETH_PENDLE.csv", timestamp_start)
write_array_to_csv(LP_profit, "data\LP_profit\stETH.csv", timestamp_start)
write_array_to_csv(LP_profit_PENDLE_YT, "data\LP_profit\stETH_PENDLE_YT.csv", timestamp_start)
write_array_to_csv(LP_profit_YT, "data\LP_profit\stETH_YT.csv", timestamp_start)